# Mongo Function

In [80]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from bson.objectid import ObjectId

In [ ]:
load_dotenv()

MONGO_URI = os.getenv('MONGO_URI')
client = MongoClient(MONGO_URI)

db = client["ImaginAI"]
collection = db["stock_data"]

## Get Home Images

In [ ]:
def GetHomeImages(collection=None,cursor=0,size=50):
    if collection is None:
        return {}
    query = {}
    documents = list(collection.find(query).skip(cursor).limit(size))
    url_list = [doc["image_url_list"][0].split("/")[-1] for doc in documents]
    next_cursor = cursor + len(url_list)
    return {"url_list":url_list,"next_cursor":next_cursor}

In [ ]:
GetHomeImages(collection,0,10)

## Search Prompt

In [ ]:
def SearchPrompt(collection=None,cursor=0,size=50,prompt=""):
    if len(prompt.split(" ")) > 3:
        return {"response" : "Too Many keywords"}
    if prompt == "":
        return {"response" : "No Keywords provided"}
    query = [
        {
            '$search': {
                'index': 'search_prompt',
                'text': {
                    'query': f'{prompt}',
                    'path': {
                        'wildcard': '*'
                    },
                    'fuzzy': {}
                }
            }
        },
        {
            '$limit': size  # Number of documents to retrieve
        },
        {
            '$skip': cursor  # Number of documents to skip
        }
    ]
    result = collection.aggregate(query)

    return [doc for doc in result]

In [ ]:
documents = SearchPrompt(collection,0,50,"car")

In [ ]:
for doc in documents:
    print(doc['prompt'])

## Search Image

In [109]:

def SearchImage(collection=None,vector=[]):
    query = [
                {
                        "$search": {
                            "index": "search_image",
                            "knnBeta": {
                            "vector": vector,
                            "path": "content_embedding",
                            "k": 5
                            }
                        }
                },
                    {
                    "$skip": 100
                    },
                    {
                    "$limit": 50
                    }
            ]
    result = collection.aggregate(query)

    return [doc for doc in result]

In [110]:
vector = collection.find({"_id":ObjectId("641d5a8b10d0c2407e98b8f6")},{"content_embedding"})[0]['content_embedding']
documents = SearchImage(collection,vector)

In [111]:
for doc in documents:
    print(doc['_id'])

641d5a8b10d0c2407e98b8f6
641d5f98dc7673f35a73d818
642d840481b26223d05d3986
642d816b81b26223d05d291c
642d82cf81b26223d05d31c9
